# Data cleaning Immobiliare.it

Importing the librares

In [428]:
import numpy as np
import pandas as pd
import re

I check the raw dataframe and the datatypes

In [429]:
df_raw = pd.read_csv('house_prices_italy.csv')
df_raw.head()

,Unnamed: 0,region,city,area,rooms,toilets,price
0,0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,2,abruzzo,Pescara,227m²,5,3+,€ 299.000
3,3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000
4,4,abruzzo,Roseto degli Abruzzi,978m²,5+,3+,€ 1.500.000


In [430]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  40000 non-null  int64 
 1   region      40000 non-null  object
 2   city        40000 non-null  object
 3   area        39975 non-null  object
 4   rooms       39050 non-null  object
 5   toilets     40000 non-null  object
 6   price       40000 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


I know that I will need to change datatypes at least on 'area' and 'price'.

Are there any null values? let's see:

In [431]:
df_raw.isnull().sum()

Unnamed: 0      0
region          0
city            0
area           25
rooms         950
toilets         0
price           0
dtype: int64

I decided to drop the first column since it does not carry any info:

In [432]:
df=df_raw.drop("Unnamed: 0", axis=1)

In [433]:
df.head()

,region,city,area,rooms,toilets,price
0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,abruzzo,Pescara,227m²,5,3+,€ 299.000
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000
4,abruzzo,Roseto degli Abruzzi,978m²,5+,3+,€ 1.500.000


I force the "area" column as 'string', if not, later it will give errors.

In [434]:
df['area'] = df['area'].astype(str)

I do not like the column names. I rename them:

In [435]:
df.rename(columns={'area':'area[m2]', 'price':'price[€]'}, inplace=True)
df.sample(10)

,region,city,area[m2],rooms,toilets,price[€]
18791,marche,San Benedetto del Tronto,70m²,3,1,€ 200.000
15988,liguria,Genova,50m²,2,1,€ 65.000
18303,marche,Fano,130m²,5,2,€ 275.000
20908,molise,Montenero di Bisaccia,106m²,4,1,€ 49.000
37070,valle-d-aosta,Courmayeur,75m²,3,2,€ 700.000
4548,campania,Trentola-Ducenta,90m²,2,2,€ 225.000
11576,friuli-venezia-giulia,Pordenone,140m²,4,2,€ 125.000
32379,trentino-alto-adige,Riva del Garda,120m²,4,3,€ 519.000
15872,liguria,Genova,95m²,5+,2,€ 299.000
4271,campania,San Nicola la Strada,135m²,4,2,€ 335.000


Regex on "area" column. Let's start cleaning from here:

In [436]:
#compilo il pattern
p = re.compile('[0-9]+')


df['area[m2]'] = df['area[m2]'].apply(lambda x: 0 if p.search(x) is None else p.search(x).group())
df.head()

,region,city,area[m2],rooms,toilets,price[€]
0,abruzzo,Pescara,89,3,1,€ 75.000
1,abruzzo,Spoltore,199,5+,3+,€ 235.000
2,abruzzo,Pescara,227,5,3+,€ 299.000
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43,2 - 4,1,da € 165.000
4,abruzzo,Roseto degli Abruzzi,978,5+,3+,€ 1.500.000


I convert 'area' column into 'Int64' dtype:

In [437]:
df['area[m2]'] = df['area[m2]'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    40000 non-null  object
 1   city      40000 non-null  object
 2   area[m2]  40000 non-null  int64 
 3   rooms     39050 non-null  object
 4   toilets   40000 non-null  object
 5   price[€]  40000 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.8+ MB


It's the turn for the "price" column:

In [438]:
df['price[€]'] = df['price[€]'].str.replace('.','', regex=False)
df['price[€]'] = df['price[€]'].str.replace('€','', regex= False)
df['price[€]']


0                      75000
1                     235000
2                     299000
3                 da  165000
4                    1500000
                ...         
39995                 890000
39996                  79000
39997                 260000
39998                 590000
39999    Prezzo su richiesta
Name: price[€], Length: 40000, dtype: object

In [439]:
p_price = re.compile('\d+')
df['price[€]'] = df['price[€]'].apply(lambda x: 0 if x.strip().isnumeric() == False else p.search(x).group())

In [440]:
df['price[€]'] = df['price[€]'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    40000 non-null  object
 1   city      40000 non-null  object
 2   area[m2]  40000 non-null  int64 
 3   rooms     39050 non-null  object
 4   toilets   40000 non-null  object
 5   price[€]  40000 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.8+ MB


The 'rooms' and 'toilets' columns are still treated as strings. As they are in this moment one could use them as categorical values. I leave it this way.

I now rename the regions into a more appropriate manner:

In [441]:
regions_dict = {'abruzzo':'Abruzzo', 'basilicata':'Basilicata', 'campania':'Campania', 'calabria':'Calabria', 'emilia-romagna':'Emilia Romagna',
       'friuli-venezia-giulia':'Friuli-Venezia Giulia', 'lazio': 'Lazio', 'liguria':'Liguria', 'lombardia':'Lombardia', 'marche':'Marche',
       'molise':'Molise', 'piemonte':'Piemonte', 'puglia':'Puglia', 'sardegna':'Sardegna', 'sicilia':'Sicilia', 'toscana':'Toscana',
       'trentino-alto-adige': 'Trentino-Alto Adige', 'umbria':'Umbria', 'valle-d-aosta':'Valle d\'Aosta', 'veneto':'Veneto'}

df.replace({'region':regions_dict}, inplace=True)
df.isna().sum()


region        0
city          0
area[m2]      0
rooms       950
toilets       0
price[€]      0
dtype: int64

In [442]:
df.sample(20)

,region,city,area[m2],rooms,toilets,price[€]
20816,Molise,Termoli,116,4,2,140000
25750,Puglia,Matino,80,3,1,69000
12156,Lazio,Roma,96,4,2,725000
21935,Molise,Campobasso,80,3,2,0
11288,Friuli-Venezia Giulia,Trieste,73,2,1,159000
18950,Marche,Civitanova Marche,69,3,1,180000
28915,Sicilia,San Giovanni la Punta,142,5,2,129000
3585,Basilicata,Grassano,65,NaN,1,0
23688,Piemonte,Nichelino,170,5,2,245000
319,Abruzzo,Avezzano,86,3,2,160000


I believe this is enough. It will be possible to load the data into a DataViz software and keep going from there with some visualizations

In [443]:
df.to_csv('house_prices_italy_cleaned.csv')